In [ ]:
pip install opencv-python numpy scikit-image


In [25]:
import cv2
import numpy as np
from skimage.filters import threshold_otsu
from skimage.morphology import remove_small_objects
from skimage.measure import label, regionprops

# Buka video
video_path = 'jogja.mp4'
cap = cv2.VideoCapture(video_path)

# Baca semua frame
frames = []
while True:
    ret, frame = cap.read()
    if not ret:
        break
    frames.append(frame)
cap.release()

if len(frames) == 0:
    print("❌ Tidak ada frame yang terbaca dari video.")
    exit()

# Konversi ke grayscale dan buat background median
gray_frames = [cv2.cvtColor(f, cv2.COLOR_BGR2GRAY) for f in frames]
background_gray = np.median(np.stack(gray_frames, axis=0), axis=0).astype(np.uint8)

# Output video
height, width, _ = frames[0].shape
out = cv2.VideoWriter('klasifikasi_kendaraan.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 20.0, (width, height))

kernel = np.ones((3, 3), np.uint8)

for idx, frame in enumerate(frames):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    diff = cv2.absdiff(gray, background_gray)

    # Otsu Threshold
    thresh_val = threshold_otsu(diff)
    _, binary = cv2.threshold(diff, thresh_val, 255, cv2.THRESH_BINARY)

    # Morfologi
    binary = cv2.dilate(binary, kernel, iterations=1)
    binary = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)
    binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)

    label_img = label(binary > 0)
    filtered = remove_small_objects(label_img, min_size=50)

    output = frame.copy()

    for region in regionprops(filtered):
        if region.area >= 300:
            minr, minc, maxr, maxc = region.bbox
            w = maxc - minc
            h = maxr - minr
            area = w * h

            # Ambil posisi vertikal tengah dari bounding box
            center_y = (minr + maxr) // 2

            # Bagi video menjadi 3 zona
            third = height // 3

            # Klasifikasi berdasarkan zona posisi vertikal
            if center_y < third:
                if area >= 14000:
                    size_label = 'Mobil'
                    color = (255, 0, 0)
                elif 3000 <= area < 9000:
                    size_label = 'Motor'
                    color = (0, 255, 255)
                else:
                    continue
            elif center_y < 2 * third:
                if area >= 60000:
                    size_label = 'Mobil'
                    color = (255, 0, 0)
                elif 1500 <= area < 20000:
                    size_label = 'Motor'
                    color = (0, 255, 255)
                else:
                    continue
            else:
                if area >= 50000:
                    size_label = 'Mobil'
                    color = (255, 0, 0)
                elif 5000 <= area < 45000:
                    size_label = 'Motor'
                    color = (0, 255, 255)
                else:
                    continue

            # Tampilkan bounding box dan label
            cv2.rectangle(output, (minc, minr), (maxc, maxr), color, 2)
            cv2.putText(output, size_label, (minc, minr - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    out.write(output)

out.release()
cv2.destroyAllWindows()

print("✅ Video selesai diproses")


✅ Video selesai diproses
